# CRITIC法
## 算法简介
该算法也是用来赋权重的一种方法。CRITIC 是Diakoulaki（1995）提出一种评价指标客观赋权方法。该方法在对指标进行权重计算时围绕两个方面进行：对比度和矛盾性。

## 案例分析
还是用一篇高引用论文为例,针对对不同银行的指标进行银行评价。现有中信、光大、浦发、招商四个银行。对它们的资产收益率、费用利润率、逾期贷款率、资产使用、自有资本率分别进行评价。数据表格如下：
![image-20220129133819170](https://cdn.jsdelivr.net/gh/GEAMING-CHN/images/blogimg/%E6%9D%82%E9%A1%B9/image-20220129133819170.png)

## 原理分析
1. 数据标准化
    每个指标的数量级不一样，需要把它们化到同一个范围内比较。指标也都需要正向化。此篇把正向化和标准化结合。
    
    设有$m$个待评对象，$n$个评价指标，可以构成数据矩阵$X=(x_{ij})_{m\times n}$
    
    设标准化后的数据矩阵元素为$x_{ij}^{\prime}$
    
    若$x_j$为负向指标（越小越优型指标），数据表格中的逾期贷款率属于此类指标。
    $$x_{ij}^{\prime}=\frac{\mathrm{max}(x_j)-x_{ij}}{\mathrm{max}(x_j)-\mathrm{min}(x_j)}$$
    若$x_j$为正向指标（越大越优型指标），数据表格中其余所有指标属于此类指标。
    $$x_{ij}^{\prime}=\frac{x_{ij}-\mathrm{min}(x_j)}{\mathrm{max}(x_j)-\mathrm{min}(x_j)}$$    
    
2. 信息承载量计算
    
    CRITIC法对指标进行权重计算时围绕两个方面进行：对比度和矛盾性
    - 对比性
        
        用标准差$\sigma_j$表示$j$项指标的对比性
        
        $\sigma_j=\sqrt{\frac{\sum_{i=1}^m\ (x_{ij}^{\prime}-\overline{x_j^{\prime}})}{m-1}}$
        
    - 矛盾性
        
        矛盾性反映的是不同指标之间的相关程度，若呈现显著正相关性，则矛盾性数值越小。设指标$j$与其余指标矛盾性的大小为$f_j$
        $$f_j=\sum_{i=1}^{m}(1-r_{ij})$$
        $r_{ij}$表示指标$i$与指标$j$之间的相关系数，在此使用的是皮尔逊相关系数，此为线性相关系数。    
    
    - 信息承载量
        设指标$j$与信息承载量为$C_j$
        $$C_j=\sigma_jf_j$$
3. 权重计算

    信息承载量越大可认为权重越大
    $$w_j=\frac{C_j}{\sum_{j=1}^nC_j}$$
    计算得分
    $$S_i=\sum_{j=1}^nw_jx_{ij}^{\prime}$$

In [1]:
import copy
import pandas as pd
import numpy as np

In [2]:
data=pd.read_excel('银行数据.xlsx')
data

,银行,资产收益率,费用利润率,逾期贷款率,资产使用,自有资本率
0,中信,0.4830,13.2682,0.0000,4.3646,5.1070
1,光大,0.4035,13.4909,39.0131,3.6151,5.5005
2,浦发,0.8979,25.7776,9.0513,4.8920,7.5342
3,招商,0.5927,16.0245,13.2935,4.4529,6.5913


In [3]:
#数据标准正向化处理
label_need=data.keys()[1:]
data1=data[label_need].values
data2=copy.deepcopy(data1)
[m,n]=data2.shape
index_all=np.arange(n)
index=[2] #负向指标位置,注意python是从0开始计数，对应位置也要相应减1
for j in index:
    d_max=max(data1[:,j])
    d_min=min(data1[:,j])
    data2[:,j]=(d_max-data1[:,j])/(d_max-d_min)
#正向指标位置    
index=np.delete(index_all,index) 
for j in index:
    d_max=max(data1[:,j])
    d_min=min(data1[:,j])
    data2[:,j]=(data1[:,j]-d_min)/(d_max-d_min)
data2

array([[0.16080097, 0.        , 1.        , 0.58696844, 0.        ],
       [0.        , 0.01780261, 0.        , 0.        , 0.16212096],
       [1.        , 1.        , 0.76799332, 1.        , 1.        ],
       [0.38268608, 0.22033831, 0.65925548, 0.65612029, 0.61152769]])

In [4]:
#信息承载量计算
#对比性
the=np.std(data2,axis=0)
data3=copy.deepcopy(data2)
#矛盾性
data3=list(map(list,zip(*data2))) #矩阵转置
r=np.corrcoef(data3)   #求皮尔逊相关系数
f=np.sum(1-r,axis=1)
#信息承载量
c=the*f

In [5]:
#计算得分
w=c/sum(c)  #计算权重
s=np.dot(data2,w)
Score=100*s/max(s) #计算得分
for i in range(0,len(Score)):
    print(f"{data['银行'][i]}银行百分制评分为:{Score[i]}")  

中信银行百分制评分为：49.70763422811035
光大银行百分制评分为：4.025015628082672
浦发银行百分制评分为：100.0
招商银行百分制评分为：58.409900390756505
